In [3]:
import pandas as pd
import os

We have updated the FUE locations in Galveston to include indoor and outdoor points. This notebook uses the three dataframes galveston_indoor_pts, galveston_outdoor_pts, and the original df for FUE_Galveston to make a single dataframe that we will use in trial 2 of our routing optimizations:
1. merges the number of pickup and drop off totes from the original dataframe to the indoor points dataframe
2. adds a column "location_type" = "indoor" to every point in the indoor dataframe 
3. subsets the outdoor dataframe to only Truck pickup points
4. adds a daily pickup value of 1 to every outdoor point
5. adds a column "location_type" = "outdoor" to every point in the outdoor dataframe 
6. capitalizes the "longitude" and "latitude" columns in the outdoor df
7. concatonates the indoor and outdoor points into a single dataframe
8. fills all "pickup_type" rows with "Truck"
9. adds the "Moody Gardens" location to the full dataframe
10. sets "Moody Garden"'s "location_type" to "depot"

In [62]:
# 1. merges the number of pickup and drop off totes from the original dataframe 
# to the indoor points dataframe
indoor = pd.read_csv("../data/galveston_indoor_pts.csv")
galveston = pd.read_csv("../data/FUE_Galveston.csv")
#subset the galveston dataframe to the columns we need
galveston_sub = galveston.loc[:, ["Name", "Weekly_Dropoff_Totes", "Daily_Pickup_Totes"]]
indoor_with_totes = hmm = pd.merge(indoor, 
                                   galveston_sub, 
                                   on = "Name", 
                                   how = 'left')

In [32]:
# 2. adds a column "location_type" = "indoor" to every point in the indoor dataframe 
indoor_with_totes.loc[:, "location_type"] = "indoor"

In [33]:
# 3. subsets the outdoor dataframe to only Truck pickup points
outdoor = pd.read_csv("../data/galveston_outdoor_pts.csv")
outdoor = outdoor.loc[outdoor.loc[:,"pickup_type"]=="Truck",:]

In [41]:
# 4. adds a daily pickup value of 1.0 and weekly dropoff value to 0.0 to every outdoor point
outdoor.loc[:, "Daily_Pickup_Totes"] = 1.0
outdoor.loc[:, "Weekly_Dropoff_Totes"] = 0.0

In [42]:
# 5. adds a column "location_type" = "outdoor" to every point in the outdoor dataframe 
outdoor.loc[:, "location_type"] = "outdoor"

In [50]:
# 6. capitalizes the "longitude" and "latitude" columns in the outdoor df
outdoor = outdoor.rename(columns={"longitude": "Longitude", "latitude": "Latitude"})

In [73]:
# 7. concatonates the indoor and outdoor points into a single dataframe
truck_service_locations = pd.concat([indoor_with_totes, outdoor])
truck_service_locations.head()

,Name,user_ratings_total,category,rating,Latitude,Longitude,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,Weekly_Dropoff_Totes,Daily_Pickup_Totes,id,pickup_type,location_type
0,Sugar Bean,476.0,cafe,4.7,29.261029,-94.870418,200,50,200,"11 Evia Main, Galveston",POINT (-94.8704183 29.2610286),Curious,High,No,High,1.0,1.0,NaN,NaN,NaN
1,MAC-IES,73.0,restaurant,5.0,29.298912,-94.791385,1500,1050,1500,"1110 23rd Street, Galveston",POINT (-94.7913851 29.2989118),Curious,High,No,Low,1.0,1.0,NaN,NaN,NaN
2,Good Dough,68.0,bakery,4.6,29.290612,-94.808531,70,35,70,"1508 39th Street, Galveston",POINT (-94.808531 29.2906119),Curious,High,No,Med,1.0,1.0,NaN,NaN,NaN
3,Porch Cafe,332.0,restaurant,4.1,29.319581,-94.750192,300,NaN,100,"1625 East Beach Drive #2A, Galveston",POINT (-94.7501916 29.3195806),Curious,High,No,Low,1.0,1.0,NaN,NaN,NaN
4,Coastal Grill,814.0,restaurant,4.6,29.308158,-94.789455,500,100,500,"1827 The Strand, Galveston",POINT (-94.789455 29.308158),Skeptical,High,No,Med,1.0,1.0,NaN,NaN,NaN


In [74]:
# 8. fills all "pickup_type" rows with "Truck", and renames the column "service_type"
truck_service_locations.loc[:, "pickup_type"] = "Truck"
truck_service_locations = truck_service_locations.rename(
                            columns={"pickup_type": "service_type"})

In [75]:
# 9. adds the "Moody Gardens" location to the full dataframe
moody_gardens = galveston.loc[(galveston.loc[:, "Name"]==
                               "Moody Gardens"), (
                            ["Name", "Longitude", "Latitude", "Daily_Pickup_Totes", "Weekly_Dropoff_Totes"])]
truck_service_locations = pd.concat([truck_service_locations, moody_gardens])
truck_service_locations.reset_index(drop=True, inplace=True)

truck_service_locations.tail()

,Name,user_ratings_total,category,rating,Latitude,Longitude,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,Weekly_Dropoff_Totes,Daily_Pickup_Totes,id,service_type,location_type
238,NaN,NaN,NaN,NaN,29.227723,-94.896043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3031.0,Truck,outdoor
239,NaN,NaN,NaN,NaN,29.210431,-94.927823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3032.0,Truck,outdoor
240,NaN,NaN,NaN,NaN,29.193525,-94.954036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3033.0,Truck,outdoor
241,NaN,NaN,NaN,NaN,29.196348,-94.956086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3034.0,Truck,outdoor
242,Moody Gardens,NaN,NaN,NaN,29.273600,-94.852300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN


In [77]:
# 10. sets "Moody Garden"'s "location_type" to "depot"
truck_service_locations.loc[(truck_service_locations.loc[:, "Name"]== "Moody Gardens"),
                            "location_type"] = "depot"
truck_service_locations.tail()

,Name,user_ratings_total,category,rating,Latitude,Longitude,weekly_high,weekly_low,best_weekly_estimate,Address,geometry,Adoption,Adoption Guess,Visited,Influence,Weekly_Dropoff_Totes,Daily_Pickup_Totes,id,service_type,location_type
238,NaN,NaN,NaN,NaN,29.227723,-94.896043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3031.0,Truck,outdoor
239,NaN,NaN,NaN,NaN,29.210431,-94.927823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3032.0,Truck,outdoor
240,NaN,NaN,NaN,NaN,29.193525,-94.954036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3033.0,Truck,outdoor
241,NaN,NaN,NaN,NaN,29.196348,-94.956086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,3034.0,Truck,outdoor
242,Moody Gardens,NaN,NaN,NaN,29.273600,-94.852300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,depot


Save the resulting dataframe.

In [82]:
truck_service_locations.to_csv("../data/galv_truck_service.csv", index=False)